In [1]:
!pip install recordlinkage

You should consider upgrading via the 'c:\users\bookw\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import recordlinkage

In [3]:
organization_dupes = pd.read_csv('new_input_file_edited.csv', index_col='Id')

In [4]:
organization_dupes.columns

Index(['address', 'canonical_name', 'city', 'country', 'country_code', 'id',
       'name', 'source', 'state', 'type', 'url'],
      dtype='object')

In [7]:
dupe_indexer = recordlinkage.Index()
dupe_indexer.sortedneighbourhood(left_on='country_code')
dupe_candidate_links = dupe_indexer.index(organization_dupes)

In [8]:
compare_dupes = recordlinkage.Compare()

compare_dupes.exact('country_code', 'country_code', label='organization_country_code')
compare_dupes.string('name',
                    'name',
                    threshold=0.80,
                     method='damerau_levenshtein',
                    label='organization_name')
compare_dupes.string('address',
                    'address',
                    threshold=0.85,
                     method='damerau_levenshtein',
                    label='organization_address')
compare_dupes.string('canonical_name',
                    'canonical_name',
                    threshold=0.85,
                    label='organization_canonical')
compare_dupes.exact('city', 'city', label='organization_city')

dupe_features = compare_dupes.compute(dupe_candidate_links, organization_dupes)

In [9]:
dupe_features.sum(axis=1).value_counts().sort_index(ascending=False)

5.0         949
4.0         100
3.0       14001
2.0      103087
1.0    34011855
0.0      494492
dtype: int64

In [10]:
potential_dupes = dupe_features[dupe_features.sum(axis=1) > 1].reset_index()
potential_dupes['Score'] = potential_dupes.loc[:, 'organization_name':'organization_city'].sum(axis=1)

In [11]:
potential_dupes

,Id_1,Id_2,organization_country_code,organization_name,organization_address,organization_canonical,organization_city,Score
0,2,1,1,0.0,0.0,1.0,0,1.0
1,3,1,1,1.0,1.0,1.0,0,3.0
2,3,2,1,0.0,0.0,1.0,0,1.0
3,4,1,1,0.0,0.0,1.0,0,1.0
4,4,2,1,1.0,1.0,1.0,0,3.0
...,...,...,...,...,...,...,...,...
118132,13908,13886,0,1.0,0.0,1.0,0,2.0
118133,13908,13888,0,1.0,0.0,1.0,0,2.0
118134,13908,13890,0,1.0,0.0,1.0,0,2.0
118135,13908,13892,0,1.0,0.0,1.0,0,2.0


In [12]:
organization_dupes.loc[11906, :]

address                                            NaN
canonical_name                                 shaehan
city                                               NaN
country                                    South Korea
country_code                                        KR
id                fd3cd261-f835-4129-8d61-9b946895a0aa
name                                           SHAEHAN
source            9b50d4d5-6c83-4ae8-8c47-5f722e35a78b
state                                              NaN
type              905e0790-81a3-4396-acb6-e715ce483f6a
url                                                NaN
Name: 11906, dtype: object

In [13]:
organization_dupes.loc[10579, :]

address                  32-68, Oehang 1-gil,Gunsan-si
canonical_name                                 shaehan
city                                         Gunsan-si
country                      South Korea (Republic of)
country_code                                        KR
id                42e2a9a9-21ed-43bc-93cc-bb5720628d53
name                                           SHAEHAN
source            9b50d4d5-6c83-4ae8-8c47-5f722e35a78b
state                                              NaN
type              905e0790-81a3-4396-acb6-e715ce483f6a
url                                                NaN
Name: 10579, dtype: object

In [14]:
organization_dupes.loc[3, :]

address           24 Anns Nagar, KPN Colony, Tirupur-641601
canonical_name                          ahillapparelexports
city                                                    NaN
country                                               India
country_code                                             IN
id                     87bb8b6e-daab-4195-9042-19ca17b96555
name                          Ahill Apparel Exports PVT LTD
source                 48aca846-b7a8-4ecb-bf24-685395f1592a
state                                                   NaN
type                   905e0790-81a3-4396-acb6-e715ce483f6a
url                                                     NaN
Name: 3, dtype: object

In [15]:
organization_dupes.loc[1, :]

address           24 Anns Nagar, KPN Colony, Tirupur-641601
canonical_name                          ahillapparelexports
city                                                    NaN
country                                               India
country_code                                             IN
id                     87bb8b6e-daab-4195-9042-19ca17b96555
name                          Ahill Apparel Exports PVT LTD
source                 48aca846-b7a8-4ecb-bf24-685395f1592a
state                                                   NaN
type                   905e0790-81a3-4396-acb6-e715ce483f6a
url                                                     NaN
Name: 1, dtype: object

In [18]:
potential_dupes.sort_values(by=['Score'], ascending=False).head(50)

,Id_1,Id_2,organization_country_code,organization_name,organization_address,organization_canonical,organization_city,Score
31319,163,71,1,1.0,1.0,1.0,1,4.0
1193,145,13,1,1.0,1.0,1.0,1,4.0
1189,144,116,1,1.0,1.0,1.0,1,4.0
1187,144,104,1,1.0,1.0,1.0,1,4.0
1185,144,92,1,1.0,1.0,1.0,1,4.0
1183,144,76,1,1.0,1.0,1.0,1,4.0
1181,144,64,1,1.0,1.0,1.0,1,4.0
1179,144,52,1,1.0,1.0,1.0,1,4.0
1177,144,36,1,1.0,1.0,1.0,1,4.0
1175,144,24,1,1.0,1.0,1.0,1,4.0


In [19]:
organization_dupes.loc[71, :]

address           22 Km Ferozepur Road 3.8 Km Rohi Nala Moza Kac...
canonical_name                                                kamal
city                                                         Lahore
country                                                    Pakistan
country_code                                                     PK
id                             3e224f14-4de0-4f2a-bb9e-86e162bea744
name                                                  KAMAL LIMITED
source                         9b50d4d5-6c83-4ae8-8c47-5f722e35a78b
state                                                           NaN
type                           905e0790-81a3-4396-acb6-e715ce483f6a
url                                                             NaN
Name: 71, dtype: object

In [20]:
organization_dupes.loc[163, :]

address           22 Km Ferozepur Road 3.8 Km Rohi Nala Moza Kac...
canonical_name                                                kamal
city                                                         Lahore
country                                                    Pakistan
country_code                                                     PK
id                             3e224f14-4de0-4f2a-bb9e-86e162bea744
name                                                  KAMAL LIMITED
source                         9b50d4d5-6c83-4ae8-8c47-5f722e35a78b
state                                                           NaN
type                           905e0790-81a3-4396-acb6-e715ce483f6a
url                                                             NaN
Name: 163, dtype: object